<a href="https://colab.research.google.com/github/Kaiziferr/Deep_Learning_Workshop/blob/master/05_workshop_binary_clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/Kaiziferr/Deep_Learning_Workshop/master/workshop_clasification_binary_multiclass/sonar_csv.csv')
data_values = data.values

In [3]:
X = data_values[:,0:60].astype('float64')
y = data_values[:,60]

In [4]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
encoded_y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Especificacion

1. Una capa oculta completamente conectada
2. Funcion de activacion ReLu
3. Capa salida una sola neurona funcion de activacion sigmoide
4. Funcion de perdida binary_crossentropy
5. Optimizer adam y Accuracy como metrica

In [5]:
def base_model():
  model = Sequential()
  model.add(Dense(60,input_dim=60,activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [6]:
estimator = KerasClassifier(build_fn=base_model, epochs = 100, batch_size = 5, verbose = 0)

In [7]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_y, cv = kfold)
print(" Linea base: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

 Linea base: 81.29% (9.80%)


# Optimizar el rendimiento con procesamiento de datos

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [12]:
estimator = []
estimator.append(('standarize', StandardScaler()))
estimator.append(('mlp', KerasClassifier(build_fn=base_model, epochs = 100, batch_size=5, verbose = 0)))

In [13]:
pipeline = Pipeline(estimator)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)

In [14]:
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 80.86% (9.90%)


# Topologia pequeña

In [15]:
def create_smaller():
    model = Sequential()
    model.add(Dense(30, input_dim = 60, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [16]:
estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs = 100, batch_size=5, verbose = 0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 84.10% (8.07%)


# Topologia grande

In [17]:
def create_larger():
    model = Sequential()
    model.add(Dense(60, input_dim = 60, activation = 'relu'))
    model.add(Dense(30, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [18]:

estimators = []
estimators.append(('standarize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs = 100, batch_size=5, verbose = 0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Modelo Estandarizado: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Modelo Estandarizado: 85.10% (6.19%)
